In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np
from sklearn.cluster import OPTICS
import time 
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import sys
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

In [ ]:
#accessing to tweets 
with open("tweet_ent.txt", "rb") as fp:   # Unpickling
  tweet_ent = pickle.load(fp)
  
with open("tweet_mng.txt", "rb") as fp:   # Unpickling
  tweet_mng = pickle.load(fp)

In [ ]:
def cleaning (text):

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   
    
    #replace consecutive non-ASCII characters with a space
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)
    
    #removing hashtag sign
    text=re.sub('#', '', text) 
    
    text=''.join(text)
   
    return text

In [ ]:
ent_hash_num=0
ent_hash_tweet=[]
for tweet in tweet_ent:
    ans=[cleaning(i)  for i in tweet.split() if i.startswith("#") ]
    if ans!=[]:
#        ans=' '.join(ans)
#        ent_hash_tweet.append(ans)
        ent_hash_tweet.extend(ans)
        ent_hash_num +=1
print(len(ent_hash_tweet))         

In [ ]:
ent_hash_tweet[0:10]

In [ ]:
mng_hash_num=0
mng_hash_tweet=[]
for tweet in tweet_mng:
    ans=[cleaning(i)  for i in tweet.split() if i.startswith("#") ]
    if ans!=[]:
#        ans=' '.join(ans)
#        mng_hash_tweet.append(ans)
        mng_hash_tweet.extend(ans)
        mng_hash_num +=1
print(len(mng_hash_tweet))        

In [ ]:
mng_hash_tweet[0:2]

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True) 

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
#for each tweet create a list of hashtags
#for each tweet, find the embedding vector for each hashtag(without #) in the tweet. and store all of a them in a list  
#Finally we have to lists: one for hashtags of tweets and one for thier embedding vectors. 
#e.g. tweet_ent_hash=[ [ hashtags of tweet 1], [hashtags of tweet 4,], ......]
#e.g. embedding_ent_hash=[[vecotos for hashtags of tweet 1], [vecotos for hashtags of tweet 4], ......]

In [ ]:
t0 = time.time()

ent_embedding=[]
ent_tokenized=[]

for text in ent_hash_tweet:
#    print(text)
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
#    print(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        out = outputs[0]
  
    token_embeddings = torch.squeeze(out, dim=0)  
    
    token_embeddings_list=token_embeddings.tolist()
    
    tokenized_text.pop(0)
    tokenized_text.pop(-1)
    token_embeddings_list.pop(0)
    token_embeddings_list.pop(-1)
    for ii in range(len(tokenized_text)-1 ,0 ,-1):
#        print(ii)
#        print(tokenized_text[ii][0:2])
        if tokenized_text[ii][0:2]=="##":
#            print(tokenized_text[ii])
            token_embeddings_list[ii-1]=token_embeddings_list[ii-1]+token_embeddings_list[ii]
            token_embeddings_list.pop(ii) # or del token_vecs_sum[ii]
            tokenized_text[ii-1]=tokenized_text[ii-1]+tokenized_text[ii][2:]
            tokenized_text.pop(ii) # or del tokenized_text[ii]

    ent_embedding.append(token_embeddings_list)
    ent_tokenized.append(tokenized_text)
#    print(ent_tokenized)
#    print()
    
    #for i, token_str in enumerate(tokenized_text):
    #    print (i, token_str)
print( time.time() - t0)

In [ ]:
t0 = time.time()
ent_full_clustering = OPTICS(min_samples=10).fit(ent_full_embedding)
#ent_full_clustering = DBSCAN(eps=3, min_samples=2).fit(ent_full_embedding)
#kmeans = KMeans(n_clusters=1000) #760s for 100 clusters , 5699 s for 1000 clusters
#ent_full_clustering = kmeans.fit(ent_full_embedding)

print( time.time() - t0)